## 第5章: 係り受け解析
日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルがai.ja.zipに収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

まずは指定のデータをダウンロードします。
Google Colaboratoryのセル上で下記のコマンドを実行すると、カレントディレクトリに対象のファイルがダウンロードされます。

In [1]:
!wget https://nlp100.github.io/data/ai.ja.zip
!unzip ai.ja.zip

--2021-12-13 01:14:42--  https://nlp100.github.io/data/ai.ja.zip
Resolving nlp100.github.io (nlp100.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17516 (17K) [application/zip]
Saving to: ‘ai.ja.zip’

ai.ja.zip           100%[===================>]  17.11K  --.-KB/s    in 0s      

2021-12-13 01:14:42 (38.4 MB/s) - ‘ai.ja.zip’ saved [17516/17516]

Archive:  ai.ja.zip
  inflating: ai.ja.txt               
  inflating: readme.ai.ja.md         


In [2]:
%%bash
# mecabのインストール
apt-get install mecab swig libmecab-dev mecab-ipadic-utf8 >/dev/null
# mecab-pythonのインストール
pip -q install mecab-python3

CaboChaおよびCaboChaの実行に必要なMeCabとCRF++をインストールします。

In [3]:
%%bash
# crfppソースファイルダウンロード
fname=CRF++-0.58
curl -sL -o ${fname}.tar.gz "https://drive.google.com/uc?export=download&id=0B4y35FiV1wh7QVR6VXJ5dWExSTQ"
tar -zxf ${fname}.tar.gz
cd ${fname}
# crfppインストール
./configure --quiet
make >/dev/null
make install >/dev/null
ldconfig
cd ..

./configure: line 7077: /usr/bin/file: No such file or directory
In file included from tagger.h:14:0,
                 from feature.cpp:12:
param.h:34:13: warning: 'Target CRFPP::{anonymous}::lexical_cast(Source) [with Target = std::__cxx11::basic_string<char>; Source = std::__cxx11::basic_string<char>]' defined but not used [-Wunused-function]
 std::string lexical_cast<std::string, std::string>(std::string arg) {
             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ar: `u' modifier ignored since `D' is the default (see `U')
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [4]:
%%bash
# cabochaソースファイルダウンロード
fname=cabocha-0.69
url="https://drive.google.com/uc?export=download&id=0B4y35FiV1wh7SDd1Q1dUQkZQaUU"
curl -sc /tmp/cookie ${url} >/dev/null
code="$(awk '/_warning_/ {print $NF}' /tmp/cookie)"
curl -sLb /tmp/cookie ${url}"&confirm=${code}" -o ${fname}.tar.bz2

# 少し時間がかかる
tar -jxf ${fname}.tar.bz2
# cabochaインストール
cd ${fname}
./configure  --quiet -with-charset=utf-8
make >/dev/null
make check >/dev/null
make install >/dev/null
ldconfig
cd ..

# cabocha-pythonのインストール
pip -q install cabocha-0.69/python/

./configure: line 7604: /usr/bin/file: No such file or directory
In file included from dep_learner.cpp:17:0:
param.h:30:13: warning: 'Target {anonymous}::lexical_cast(Source) [with Target = std::__cxx11::basic_string<char>; Source = std::__cxx11::basic_string<char>]' defined but not used [-Wunused-function]
 std::string lexical_cast<std::string, std::string>(std::string arg) {
             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
In file included from libcabocha.cpp:18:0:
param.h:30:13: warning: 'Target {anonymous}::lexical_cast(Source) [with Target = std::__cxx11::basic_string<char>; Source = std::__cxx11::basic_string<char>]' defined but not used [-Wunused-function]
 std::string lexical_cast<std::string, std::string>(std::string arg) {
             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
normalizer.cpp: In static member function 'static void CaboCha::Normalizer::normalize(int, const char*, size_t, std::__cxx11::string*)':
normalizer.cpp:113:15: warning: offset '-1' outside bounds of con

In [5]:
# 結果確認
!mecab -v
!pip show mecab-python3
!cabocha -v
!pip show cabocha-python

mecab of 0.996

Name: mecab-python3
Version: 1.0.4
Summary: Python wrapper for the MeCab morphological analyzer for Japanese
Home-page: https://github.com/SamuraiT/mecab-python3
Author: None
Author-email: None
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: 
cabocha of 0.69
Name: cabocha-python
Version: 0.69
Summary: UNKNOWN
Home-page: UNKNOWN
Author: None
Author-email: None
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: 


係り受け解析を行います。
以下のコマンドを実行することにより、ai.ja.txtを係り受け解析した結果が、ai.ja.txt.cabochaとして出力されます。

In [6]:
!cabocha -f1 -o ai.ja.txt.cabocha ai.ja.txt

出力結果を確認します。

In [7]:
# 行数の確認
!wc -l ./ai.ja.txt.cabocha

11744 ./ai.ja.txt.cabocha


In [8]:
# 先頭15行の確認
!head -15 ./ai.ja.txt.cabocha

* 0 -1D 1/1 0.000000
人工	名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー
知能	名詞,一般,*,*,*,*,知能,チノウ,チノー
EOS
EOS
* 0 17D 1/1 0.388993
人工	名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー
知能	名詞,一般,*,*,*,*,知能,チノウ,チノー
* 1 17D 2/3 0.613549
（	記号,括弧開,*,*,*,*,（,（,（
じん	名詞,一般,*,*,*,*,じん,ジン,ジン
こうち	名詞,一般,*,*,*,*,こうち,コウチ,コーチ
のう	助詞,終助詞,*,*,*,*,のう,ノウ,ノー
、	記号,読点,*,*,*,*,、,、,、
、	記号,読点,*,*,*,*,、,、,、


#### 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [9]:
class Morph:
    def __init__(self, dc):
        self.surface = dc['surface']
        self.base = dc['base']
        self.pos = dc['pos']
        self.pos1 = dc['pos1']


def parse_cabocha(block):
    res = []
    for line in block.split('\n'):
        if line == '':
            return res
        elif line[0] == '*':
            continue
        (surface, attr) = line.split('\t')
        attr = attr.split(',')
        lineDict = {
            'surface': surface,
            'base': attr[6],
            'pos': attr[0],
            'pos1': attr[1]
        }
        res.append(Morph(lineDict))

In [10]:
filename = './ai.ja.txt.cabocha'
with open(filename, mode='rt', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]
for m in blocks[2]:
    print(vars(m))

{'surface': '『', 'base': '『', 'pos': '記号', 'pos1': '括弧開'}
{'surface': '日本', 'base': '日本', 'pos': '名詞', 'pos1': '固有名詞'}
{'surface': '大', 'base': '大', 'pos': '接頭詞', 'pos1': '名詞接続'}
{'surface': '百科全書', 'base': '百科全書', 'pos': '名詞', 'pos1': '一般'}
{'surface': '(', 'base': '*', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': 'ニッポニカ', 'base': '*', 'pos': '名詞', 'pos1': '一般'}
{'surface': ')』', 'base': '*', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': 'の', 'base': 'の', 'pos': '助詞', 'pos1': '連体化'}
{'surface': '解説', 'base': '解説', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': 'で', 'base': 'で', 'pos': '助詞', 'pos1': '格助詞'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': '情報', 'base': '情報', 'pos': '名詞', 'pos1': '一般'}
{'surface': '工学', 'base': '工学', 'pos': '名詞', 'pos1': '一般'}
{'surface': '者', 'base': '者', 'pos': '名詞', 'pos1': '接尾'}
{'surface': '・', 'base': '・', 'pos': '記号', 'pos1': '一般'}
{'surface': '通信', 'base': '通信', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': '工学', 'base': '工学', 'pos': '名詞', 'pos1'

#### 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

In [11]:
class Morph:
    def __init__(self, dc):
        self.surface = dc['surface']
        self.base = dc['base']
        self.pos = dc['pos']
        self.pos1 = dc['pos1']


class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs    # 形態素（Morphオブジェクト）のリスト
        self.dst = dst          # 係り先文節インデックス番号
        self.srcs = []          # 係り元文節インデックス番号のリスト


def parse_cabocha(block):
    def check_create_chunk(tmp):
        if len(tmp) > 0:
            c = Chunk(tmp, dst)
            res.append(c)
            tmp = []
        return tmp

    res = []
    tmp = []
    dst = None
    for line in block.split('\n'):
        if line == '':
            tmp = check_create_chunk(tmp)
        elif line[0] == '*':
            dst = line.split(' ')[2].rstrip('D')
            tmp = check_create_chunk(tmp)
        else:
            (surface, attr) = line.split('\t')
            attr = attr.split(',')
            lineDict = {
                'surface': surface,
                'base': attr[6],
                'pos': attr[0],
                'pos1': attr[1]
            }
            tmp.append(Morph(lineDict))

    for i, r in enumerate(res):
        res[int(r.dst)].srcs.append(i)
    return res

In [12]:
filename = './ai.ja.txt.cabocha'
with open(filename, mode='rt', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]
for m in blocks[7]:
    print([mo.surface for mo in m.morphs], m.dst, m.srcs)

['一方', '、'] 2 []
['計算', '知能'] 12 []
['（', 'CI', '）', 'は'] 4 [0]
['開発', 'や'] 5 []
['学習', 'を'] 6 [2]
['繰り返す'] 8 [3]
['こと', 'を'] 8 [4]
['基本', 'と'] 9 []
['し', 'て', 'いる'] 12 [5, 6]
['（', '例えば', '、'] 12 [7]
['パラメータ', '調整', '、'] 12 []
['コネクショニズム', 'の'] 21 []
['システム', '）', '。'] 16 [1, 8, 9, 10]
['学習', 'は'] 15 []
['経験', 'に'] 16 []
['基づく'] 21 [13]
['手法', 'で', 'あり', '、'] 18 [12, 14]
['非', '記号', '的', 'AI', '、'] 20 []
['美しく', 'ない'] 20 [16]
['AI', '、'] 21 []
['ソフトコンピューティング', 'と'] 26 [17, 18]
['関係', 'し', 'て', 'いる', '。'] 23 [11, 15, 19]
['その'] 26 []
['手法', 'として', 'は', '、'] 25 [21]
['以下', 'の'] 26 []
['もの', 'が'] -1 [23]
['ある', '。'] -1 [20, 22, 24, 25, 26]


#### 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [13]:
filename = './ai.ja.txt.cabocha'
with open(filename, mode='rt', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

for b in blocks:
    for m in b:
        if int(m.dst) > -1:
            print(''.join([mo.surface if mo.pos != '記号' else '' for mo in m.morphs]),
                  ''.join([mo.surface if mo.pos != '記号' else '' for mo in b[int(m.dst)].morphs]), sep='\t')

人工知能	語
じんこうちのう	エーアイとは
AI	語
エーアイとは	という
計算	道具を
という	道具を
概念と	という
コンピュータ	道具を
という	用いて
道具を	研究する
用いて	研究する
知能を	計算機科学
研究する	の
計算機科学	一分野を
の	指す
一分野を	語
指す	研究分野とも
語	推論
言語の	推論
理解や	問題解決などの
推論	知的行動を
問題解決などの	代わって
知的行動を	代わって
人間に	行わせる
代わって	行わせる
コンピューターに	技術または
行わせる	研究分野とも
技術または	コンピュータによる
計算機	情報処理システムの
コンピュータによる	情報処理システムの
知的な	実現に関する
情報処理システムの	実現に関する
設計や	研究分野とも
実現に関する	される
日本大百科全書(ニッポニカ)』の	述べている
解説で	佐藤理史は
情報工学者通信工学者の	述べている
佐藤理史は	述べている
人間の	実現する
知的能力を	実現する
コンピュータ上で	技術ソフトウェアコンピュータシステム
実現する	技術ソフトウェアコンピュータシステム
様々な	ある
技術ソフトウェアコンピュータシステム	ある
応用例は	機械翻訳かな漢字変換構文解析等
自然言語処理	専門家の
機械翻訳かな漢字変換構文解析等	推論判断を
専門家の	模倣する
推論判断を	エキスパートシステム
模倣する	画像認識等が
エキスパートシステム	解析して
画像データを	検出抽出したりする
解析して	パターンを
特定の	検出抽出したりする
パターンを	画像認識等が
検出抽出したりする	ある
画像認識等が	命名された
ある	命名された
1956年に	命名された
ダートマス会議で	命名された
ジョンマッカーシーにより	使われている
命名された	使われている
現在では	用いた
記号処理を	知能の
用いた	記述を
知能の	する
記述を	する
主体と	研究での
する	研究での
情報処理や	アプローチという
研究での	意味あいでも
アプローチという	使われている
意味あいでも	ある
使われている	制御システムや
家庭用電気機械器具の	思考ルーチンも
制御システムや	思考ルーチンも
ゲームソフトの	呼ばれる
思考ルーチンも	呼ばれる
こう	ことも
呼ばれる	ある
プログラミング言語による	カウンセラーを
と

#### 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [14]:
filename = './ai.ja.txt.cabocha'
with open(filename, mode='rt', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

for b in blocks:
    for m in b:
        if int(m.dst) > -1:
            pre_text = ''.join([mo.surface if mo.pos != '記号' else '' for mo in m.morphs])
            pre_pos = [mo.pos for mo in m.morphs]
            post_text = ''.join([mo.surface if mo.pos != '記号' else '' for mo in b[int(m.dst)].morphs])
            post_pos = [mo.pos for mo in b[int(m.dst)].morphs]
            if '名詞' in pre_pos and '動詞' in post_pos:
                print(pre_text, post_text, sep='\t')

道具を	研究する
問題解決などの	代わって
知的行動を	代わって
人間に	行わせる
実現に関する	される
日本大百科全書(ニッポニカ)』の	述べている
情報工学者通信工学者の	述べている
佐藤理史は	述べている
人間の	実現する
知的能力を	実現する
様々な	ある
技術ソフトウェアコンピュータシステム	ある
専門家の	模倣する
エキスパートシステム	解析して
画像データを	検出抽出したりする
特定の	検出抽出したりする
検出抽出したりする	ある
画像認識等が	命名された
1956年に	命名された
ダートマス会議で	命名された
ジョンマッカーシーにより	使われている
命名された	使われている
現在では	用いた
知能の	する
記述を	する
アプローチという	使われている
意味あいでも	ある
ゲームソフトの	呼ばれる
思考ルーチンも	呼ばれる
模倣した	出されるが
プログラム	出されるが
人工無脳が	出されるが
引き合いに	困難視されている
専門家の	させようという
研究情報処理システムの	困難視されている
実現は	持つ
人間が	持つ
常識の	なり
記述が	なり
問題と	困難視されている
実用への	困難視されている
利用が	ある
実現への	知られているが
ニューラルネットワークなどのような	知られているが
アプローチも	ある
(GoodOldFashionedAI)との	ある
記述の	ある
記号的明示性に	集めた
その後	集めた
サポートベクターマシンが	集めた
注目を	ある
自らの	行う
経験を	行う
元に	行う
強化学習という	ある
レイカーツワイルという	実装するという
言葉通り	表現し
知性を	表現し
機械的に	実装するという
ビッグデータの	行った
一過性の	超えて
流行を	浸透して
社会に	行った
浸透して	なった
2016年から	導入した
2017年にかけて	導入した
世界トップクラスの	破り
プレイヤーも	なった
麻雀では	到達するなど
MicrosoftSuphx(SuperPhoenix)」が	到達するなど
AIとして	到達するなど
十段に	なった
時代の	なった
第２次人工知能ブームでの	呼ばれ
人工知能は	呼ばれ
機械学習と	ある
以下のような	ある
開発や	繰り返す
ことを	している
コネクショニズムの	関係している
学習は	基づく
AI	関係し

#### 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，Graphviz等を用いるとよい．

係り元と係り先の文節のペアを作成し、pydotのgraph_from_edgesに渡すことで、グラフを作成しています。
なお、表層形そのままでは1文内に同じ文字列の文節が複数回出てきた場合に区別できないため、末尾にIDを付与して表示しています。



In [15]:
# 日本語表示用フォントのインストール
!apt install fonts-ipafont-gothic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-ipafont-mincho
The following NEW packages will be installed:
  fonts-ipafont-gothic fonts-ipafont-mincho
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 8,251 kB of archives.
After this operation, 28.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-ipafont-gothic all 00303-18ubuntu1 [3,526 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-ipafont-mincho all 00303-18ubuntu1 [4,725 kB]
Fetched 8,251 kB in 1s (6,523 kB/s)
Selecting previously unselected package fonts-ipafont-gothic.
(Reading database ... 156191 files and directories currently installed.)
Preparing to unpack .../fonts-ipafont-gothic_00303-18ubuntu1_all.deb ...
Unpacking fonts-ipafont-gothic (00303-18ubuntu1) ...
Selecting previously unselected package fonts-ip

In [16]:
import pydot

filename = './ai.ja.txt.cabocha'
with open(filename, mode='rt', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

pairs = []
target = blocks[7]
for m in target:
    if int(m.dst) > -1:
        pre_text = ''.join([mo.surface if mo.pos != '記号' else '' for mo in m.morphs])
        post_text = ''.join([mo.surface if mo.pos != '記号' else '' for mo in target[int(m.dst)].morphs])
        pairs.append([pre_text, post_text])

print(pairs)
g = pydot.graph_from_edges(pairs)
g.write_png('./ans44.png', prog='dot')

[['一方', 'CIは'], ['計算知能', 'システム'], ['CIは', '学習を'], ['開発や', '繰り返す'], ['学習を', 'ことを'], ['繰り返す', 'している'], ['ことを', 'している'], ['基本と', '例えば'], ['している', 'システム'], ['例えば', 'システム'], ['パラメータ調整', 'システム'], ['コネクショニズムの', '関係している'], ['システム', '手法であり'], ['学習は', '基づく'], ['経験に', '手法であり'], ['基づく', '関係している'], ['手法であり', '美しくない'], ['非記号的AI', 'ソフトコンピューティングと'], ['美しくない', 'ソフトコンピューティングと'], ['AI', '関係している'], ['ソフトコンピューティングと', 'ある'], ['関係している', '手法としては'], ['その', 'ある'], ['手法としては', 'ものが'], ['以下の', 'ある']]


#### 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

* 動詞を含む文節において，最左の動詞の基本形を述語とする
* 述語に係る助詞を格とする
* 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．
* コーパス中で頻出する述語と格パターンの組み合わせ
* 「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

In [17]:
filename = './ai.ja.txt.cabocha'
with open(filename, mode='rt', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

for b in blocks:
    for m in b:
        if len(m.srcs) > 0:
            pre_morphs = [b[int(s)].morphs for s in m.srcs]
            pre_morphs = [list(filter(lambda x: '助詞' in x.pos, pm)) for pm in pre_morphs]
            pre_surface = [[p.surface for p in pm] for pm in pre_morphs]
            pre_surface = list(filter(lambda x: x != [], pre_surface))
            pre_surface = [p[0] for p in pre_surface]
            post_base = [mo.base for mo in m.morphs]
            post_pos = [mo.pos for mo in m.morphs]
            if len(pre_surface) > 0 and '動詞' in post_pos:
                print(post_base[0], ' '.join(pre_surface), sep='\t')

用いる	という
研究	を て
指す	の
代わる	など を
行う	に て
する	に関する と
述べる	の の は の て
実現	の を
模倣	の
検出	を の
ある	たり
命名	が に で
用いる	で
する	の を
使う	により という
呼ぶ	の も
ある	でも も
模倣	という
出す	が
する	が の
呼ぶ	という
持つ	は が
なる	の が
困難	に の と へ
知る	へ など
ある	が も と の
集める	に が
行う	の を に
ある	を という
表現	を
実装	という に
超える	の
浸透	を て
行く	の に
導入	から にかけて
破る	の
到達	で が として
なる	て も に の と
呼ぶ	で は
ある	と の が
繰り返す	や
する	を
基づく	は
関係	の
ある	と の が
生成	も て の を を や を通して
上回る	の
行う	を の が
でる	の
活躍	に て が て
だす	は など で
加える	て
見せる	で て に の
広がる	て を の
行う	という
始まる	を が て
指摘	も て て
試みる	が
なる	の と て
提唱	の
製作	は の
行う	を と の を
出版	は
もたらす	と を に
内在	は
題する	の
発表	と を
築く	を に の を
出る	に と
作り出す	が の という
開発	を は
可能	に関する
導入	を に として は
構築	を
行う	は
使用	の として
創立	として
示す	の に は の
出版	は
示す	と を の
開発	を を は
構築	における
示す	を を の
呼ぶ	の
ある	を
走行	を
開発	の を
使う	に は によって に
提唱	に が を
上げる	の が
匹敵	は
開発	に
打つ	に
敗れる	を で を の を に の の
スケジューリング	の において
使う	を のに
上回る	を の
明らか	が
啓蒙	で が
発生	に に
指摘	を も の が
適す	に対して
適用	が の
行う	を
開始	が
行う	の により に
提唱	が
付加	の に
行う	を て に
規格	を
表現	における
含む	の
流行	から
分かる	て
完了	に
見合う	にとって
見出せる	だけ
する	が が て から も は て
流行	の に
直面	が
激変	に の
終焉	が は は
基づく	に
提案	が
する	に を
立ち

#### 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

* 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
* 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

In [18]:
filename = './ai.ja.txt.cabocha'
with open(filename, mode='rt', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

for b in blocks:
    for m in b:
        if len(m.srcs) > 0:
            pre_morphs = [b[int(s)].morphs for s in m.srcs]
            pre_morphs_filtered = [list(filter(lambda x: '助詞' in x.pos, pm)) for pm in pre_morphs]
            pre_surface = [[p.surface for p in pm] for pm in pre_morphs_filtered]
            pre_surface = list(filter(lambda x: x != [], pre_surface))
            pre_surface = [p[0] for p in pre_surface]
            post_base = [mo.base for mo in m.morphs]
            post_pos = [mo.pos for mo in m.morphs]
            if len(pre_surface) > 0 and '動詞' in post_pos:
                pre_text = list(filter(lambda x: '助詞' in [p.pos for p in x], pre_morphs))
                pre_text = [''.join([p.surface for p in pt]) for pt in pre_text]
                print(post_base[0], ' '.join(pre_surface), ' '.join(pre_text), sep='\t')

用いる	という	（）』という
研究	を て	道具を 用いて
指す	の	（）の
代わる	など を	問題解決などの 知的行動を
行う	に て	人間に 代わって
する	に関する と	実現に関する 研究分野」とも
述べる	の の は の て	『日本大百科全書(ニッポニカ)』の 情報工学者・通信工学者の 佐藤理史は 次のように 述べている。
実現	の を	人間の 知的能力を
模倣	の	専門家の
検出	を の	画像データを 特定の
ある	たり	検出・抽出したりする
命名	が に で	画像認識等が 1956年に ダートマス会議で
用いる	で	現在では、
する	の を	知能の 記述を
使う	により という	ジョン・マッカーシーにより アプローチという
呼ぶ	の も	ゲームソフトの 思考ルーチンも
ある	でも も	意味あいでも ことも
模倣	という	「」という
出す	が	（人工無脳）が
する	が の	出されるが、 専門家の
呼ぶ	という	させようという
持つ	は が	実現は、 人間が
なる	の が	常識の 記述が
困難	に の と へ	引き合いに 研究・情報処理システムの 問題と 実用への
知る	へ など	実現への 「ニューラルネットワーク」などのような
ある	が も と の	利用が アプローチも (GoodOldFashionedAI)との 記述の
集める	に が	記号的明示性に 「サポートベクターマシン」が
行う	の を に	自らの 経験を 元に
ある	を という	注目を 強化学習という
表現	を	知性を
実装	という に	（レイ・カーツワイル）」という 機械的に
超える	の	一過性の
浸透	を て	流行を 超えて
行く	の に	ビッグデータの 社会に
導入	から にかけて	2016年から 2017年にかけて、
破る	の	世界トップクラスの
到達	で が として	麻雀では 「MicrosoftSuphx(SuperPhoenix)」が AIとして
なる	て も に の と	浸透して プレイヤーも 十段に 時代の 最先端技術と
呼ぶ	で は	第２次人工知能ブームでの 人工知能は
ある	と の が	機械学習と 以下のような ものが
繰り返す	や	開発や
する	を	ことを
基づく	は	学習は
関係	の	コネクショニズムの
ある	と の が	ソフトコンピューティングと 以下の ものが
生

## 47. 機能動詞構文のマイニング
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

- 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
- 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

- コーパス中で頻出する述語（サ変接続名詞+を+動詞）
- コーパス中で頻出する述語と助詞パターン

In [19]:
filename = './ai.ja.txt.cabocha'
with open(filename, mode='rt', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

with open('./ans47.txt', mode='w') as f:
    for b in blocks:
        for i, m in enumerate(b):
            if 'サ変接続' in [s.pos1 for s in m.morphs] and 'を' in [s.surface for s in m.morphs] and i + 1 < len(b) and b[i + 1].morphs[0].pos == '動詞':
                text = ''.join([s.surface for s in m.morphs]) + b[i + 1].morphs[0].base
                if len(m.srcs) > 0:
                    pre_morphs = [b[int(s)].morphs for s in m.srcs]
                    pre_morphs_filtered = [list(filter(lambda x: '助詞' in x.pos, pm)) for pm in pre_morphs]
                    pre_surface = [[p.surface for p in pm] for pm in pre_morphs_filtered]
                    pre_surface = list(filter(lambda x: x != [], pre_surface))
                    pre_surface = [p[0] for p in pre_surface]
                    pre_text = list(filter(lambda x: '助詞' in [p.pos for p in x], pre_morphs))
                    pre_text = [''.join([p.surface for p in pt]) for pt in pre_text]
                    if len(pre_surface) > 0:
                        f.writelines('\t'.join([text, ' '.join(pre_surface), ' '.join(pre_text)]))
                        f.write('\n')

In [20]:
!cat ./ans47.txt | cut -f 1 | sort | uniq -c | sort -nr | head -n 10

      2 注目を集める
      1 運用をめぐる
      1 進化を見せる
      1 認識能力を持つ
      1 話をする
      1 行動型システムを用いる
      1 自律無人艇を使う
      1 精度改善を果たす
      1 禁止を求める
      1 研究を進める


In [21]:
!cat ./ans47.txt | cut -f 1,2 | sort | uniq -c | sort -nr | head -n 10

      2 注目を集める	の
      1 運用をめぐる	は
      1 進化を見せる	において
      1 認識能力を持つ	の
      1 話をする	と
      1 行動型システムを用いる	から
      1 自律無人艇を使う	に
      1 精度改善を果たす	が から
      1 禁止を求める	が
      1 研究を進める	て


## 48. 名詞から根へのパスの抽出
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

- 各文節は（表層形の）形態素列で表現する
- パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する

In [22]:
filename = './ai.ja.txt.cabocha'
with open(filename, mode='rt', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

for b in blocks:
    for m in b:
        text = []
        if '名詞' in [s.pos for s in m.morphs] and int(m.dst) != -1:
            current_chunk = m
            text.append(''.join([m.surface for m in current_chunk.morphs]))
            next_chunk = b[int(current_chunk.dst)]
            while int(current_chunk.dst) != -1:
                text.append(''.join([m.surface for m in next_chunk.morphs]))
                current_chunk = next_chunk
                next_chunk = b[int(next_chunk.dst)]
            print(*text, sep=' -> ')

人工知能 -> 語。 -> 推論、 -> 知的行動を -> 代わって -> 行わせる -> 研究分野」とも
（じんこうちのう、、 -> 〈エーアイ〉）とは、 -> （）』という -> 道具を -> 研究する -> （）の -> 指す -> 研究分野」とも
AI -> 語。 -> 推論、 -> 知的行動を -> 代わって -> 行わせる -> 研究分野」とも
〈エーアイ〉）とは、 -> （）』という -> 道具を -> 研究する -> （）の -> 指す -> 研究分野」とも
「『計算 -> 道具を -> 研究する -> （）の -> 指す -> 研究分野」とも
概念と -> （）』という -> 用いて -> 研究する -> （）の -> 指す -> 研究分野」とも
『コンピュータ -> 道具を -> 研究する -> （）の -> 指す -> 研究分野」とも
道具を -> 研究する -> （）の -> 指す -> 研究分野」とも
『知能』を -> 計算機科学 -> 一分野」を -> 語。 -> 推論、 -> 知的行動を -> 代わって -> 行わせる -> 研究分野」とも
研究する -> （）の -> 指す -> 研究分野」とも
計算機科学 -> 一分野」を -> 語。 -> 推論、 -> 知的行動を -> 代わって -> 行わせる -> 研究分野」とも
一分野」を -> 語。 -> 推論、 -> 知的行動を -> 代わって -> 行わせる -> 研究分野」とも
語。 -> 推論、 -> 知的行動を -> 代わって -> 行わせる -> 研究分野」とも
「言語の -> 推論、 -> 知的行動を -> 代わって -> 行わせる -> 研究分野」とも
理解や -> 問題解決などの -> 代わって -> 行わせる -> 研究分野」とも
推論、 -> 知的行動を -> 代わって -> 行わせる -> 研究分野」とも
問題解決などの -> 代わって -> 行わせる -> 研究分野」とも
知的行動を -> 代わって -> 行わせる -> 研究分野」とも
人間に -> 行わせる -> 研究分野」とも
コンピューターに -> 技術」、または、 -> （コンピュータ）による -> 情報処理システムの -> 実現に関する -> される。
技術」、または、 -> （コンピュータ）による -

## 49. 名詞間の係り受けパスの抽出
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がiiとjj（ii<jj）のとき，係り受けパスは以下の仕様を満たすものとする．

- 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する
- 文節iiとjjに含まれる名詞句はそれぞれ，XとYに置換する

また，係り受けパスの形状は，以下の2通りが考えられる．

- 文節iiから構文木の根に至る経路上に文節jjが存在する場合: 文節iiから文節jjのパスを表示
- 上記以外で，文節iiと文節jjから構文木の根に至る経路上で共通の文節kkで交わる場合: 文節iiから文節kkに至る直前のパスと文節jjから文節kkに至る直前までのパス，文節kkの内容を” | “で連結して表示

In [23]:
def convert(s):
    pl, nl = [], [c for c in s if '名詞' in [m.pos for m in c.morphs]]
    for i in range(len(nl) - 1):
        st1 = [''.join([m.surface if m.pos != '名詞' else 'X' for m in nl[i].morphs])]
        for e in nl[i + 1:]:
            dst, p = nl[i].dst, []
            st2 = [''.join([m.surface if m.pos != '名詞' else 'Y' for m in e.morphs])]
            while int(dst) != -1 and dst != s.index(e):
                p.append(s[int(dst)])
                dst = s[int(dst)].dst
            if len(p) < 1 or p[-1].dst != -1:
                mid = [''.join([m.surface for m in c.morphs if m.pos != '記号']) for c in p]
                pl.append(st1 + mid + ['Y'])
            else:
                mid, dst = [], e.dst
                while not s[int(dst)] in p:
                    mid.append(''.join([m.surface for m in s[int(dst)].morphs if m.pos != '記号']))
                    dst = s[int(dst)].dst
                ed = [''.join([m.surface for m in s[int(dst)].morphs if m.pos != '記号'])]
                pl.append([st1, st2 + mid, ed])
    return pl

In [24]:
filename = './ai.ja.txt.cabocha'
with open(filename, mode='rt', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

for b in blocks:
    pl = (convert(b))
    for p in pl:
        if isinstance(p[0], str):
            print(' -> '.join(p))
        else:
            print(p[0][0], ' -> '.join(p[1]), p[2][0], sep=' | ')

Output hidden; open in https://colab.research.google.com to view.